### The provided code segment consists of a series of import statements for essential libraries used in deep learning and image processing. It includes TensorFlow, PIL, glob, and pandas. Additionally, there are commented-out lines of code to mount the Google Drive to access stored files.

In [ ]:
# Import the necessary libraries
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
from glob import glob
import pandas as pd

# # Mount your Google Drive
# from google.colab import drive
# drive.mount('/content/gdrive')



### This code segment processes a collection of JPG image files by flattening them into one-dimensional arrays and organizing them into a Pandas DataFrame. It outlines the setup for training an LSTM neural network model on this data.

In [ ]:
import pandas as pd
# from google.colab import drive
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Set the path to the folder containing the jpg files.
folder_path = 'sentinel_2_l1c_toImages'

# Select all the jpg files in the folder.
jpg_files = []
for file in os.listdir(folder_path):
    if file.endswith(".jpg"):
        jpg_files.append(os.path.join(folder_path, file))

# Flatten the jpg files.
flattened_jpg_files = []
for jpg_file in jpg_files:
    image = Image.open(jpg_file)
    image_array = np.array(image)
    flattened_jpg_files.append(image_array.flatten())

# Reshape the flattened jpg files to have two dimensions.
flattened_jpg_files = np.reshape(flattened_jpg_files, (len(flattened_jpg_files), -1))

# Create a Pandas DataFrame from the flattened jpg files.
df = pd.DataFrame(flattened_jpg_files)
df = np.transpose(df)
df = np.expand_dims(df, axis=2)

# Set the path to the folder containing the test jpg files.
test_directory = 'test_images'

# Split the DataFrame into a training set and a test set.
training_set = df[:int(0.8 * len(df))]
test_set = df[int(0.8 * len(df)):]

# # Transpose the training set.
# training_set = np.transpose(training_set)
# test_set = np.transpose(test_set)

# # Reshape the transposed training set to have three dimensions.
# training_set = np.expand_dims(training_set, axis=2)
# test_set = np.expand_dims(test_set, axis=2)

# Train an LSTM model on the training set.
model = Sequential()
model.add(LSTM(128, input_shape=(training_set.shape[1], training_set.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
history = model.fit(training_set, np.zeros(training_set.shape[0]), epochs=10, batch_size= 1024)


Epoch 1/10
615/615 [==============================] - 26s 39ms/step - loss: 0.0026
Epoch 2/10
615/615 [==============================] - 24s 38ms/step - loss: 7.9433e-06
Epoch 3/10
615/615 [==============================] - 23s 38ms/step - loss: 3.2250e-06
Epoch 4/10
615/615 [==============================] - 23s 37ms/step - loss: 2.5930e-06
Epoch 5/10
615/615 [==============================] - 24s 38ms/step - loss: 4.8411e-06
Epoch 6/10
615/615 [==============================] - 23s 38ms/step - loss: 4.9191e-06
Epoch 7/10
615/615 [==============================] - 23s 38ms/step - loss: 6.1925e-06
Epoch 8/10
615/615 [==============================] - 25s 40ms/step - loss: 8.2359e-06
Epoch 9/10
615/615 [==============================] - 25s 41ms/step - loss: 8.3202e-06
Epoch 10/10
615/615 [==============================] - 25s 40ms/step - loss: 8.1672e-06


### This code snippet evaluates the LSTM neural network model on a test dataset and prints the test loss. This serves as an indicator of the model's performance on unseen data.

In [ ]:
# Evaluate the LSTM model on the test set.
test_loss = model.evaluate(test_set, np.zeros(test_set.shape[0]))
print("Test loss:", test_loss)

4916/4916 [==============================] - 61s 12ms/step - loss: 5.2572e-07
Test loss: 5.257240331957291e-07


### In this code, the pre-trained model, 'NVDI_LSTM.h5' is loaded. A random image with a specific shape (1, 97, 1) is created to match the expected input shape of the model. The model then makes a prediction on this tensor, and the resulting prediction is printed to the console.

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the model.
model = load_model('NVDI_LSTM.h5')

# Create a preprocessed image tensor.
image_tensor = np.random.rand(1, 97, 1)  # Adjust the shape to match the expected input shape.

# Make a prediction on the tensor.
prediction = model.predict(image_tensor)

# Print the prediction.
print(prediction)


1/1 [==============================] - 0s 362ms/step
[[-0.00325889]]
